In [ ]:
# import essential files from /src folder
import sys
import os

sys.path.append(os.path.abspath('../src'))

In [ ]:
# for processing
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from hog import preprocessing_split, preprocessing_full
import matplotlib.pyplot as plt
from utils import show_plot, transform_data

# for saving the model
import joblib

In [ ]:
# Load the model
ker = 'poly'  # kernel type, change to 'linear', 'rbf', 'sigmoid' or 'poly' for different kernels
deg = 3
gam = 0.1
c = 75
coef = 1
date = '02082025'   #ddmmyyyy: date of the model
time = '095138'   #hhmmss: time of the model

model = joblib.load(f'../models/kernel-{ker}_deg-{deg}_gamma-{gam}_C-{c}_coef0-{coef}_{date}_{time}.pkl')

In [ ]:
# Load the dataset
path = "D:/GitHub/FER-with-HOG-and-SVM/datasets/train/sample_1"     # path to dataset, change your path here
data_train, label_train, data_test, label_test = preprocessing_split(path)
X_train , y_train = transform_data(data_train, label_train)
X_test , y_test = transform_data(data_test, label_test)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
rp = metrics.classification_report(y_test, prediction, target_names=['anger', 'happy', 'neutral', 'sad', 'suprise'],output_dict=True)
rp = pd.DataFrame(rp).transpose()
rp.drop("support",axis=1,inplace=True)
rp.drop("macro avg",axis=0,inplace=True)
rp.drop("weighted avg",axis=0,inplace=True)
rp.drop("accuracy",axis=0,inplace=True)
rp = round(rp.iloc[0:]*100,2)
rp

# # note on metrics:
# # precision: how often the model is correct when it predicts a class (true positives / (true positives + false positives))
# high value => when model predicts a class, it is likely to be correct
# # recall: how often the model predicts a class when it is actually that class (true positives / (true positives + false negatives))
# high value => when the class is present, the model is likely to predict it
# # f1: harmonic mean of precision and recall, useful when you want to balance both metrics

In [ ]:
show_plot("Precision",rp.precision)
show_plot("Recall",rp.recall)
show_plot("F1 Score", rp['f1-score'])

In [ ]:
confusion_matrix = confusion_matrix(y_test, prediction)
confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
acc_per_class = confusion_matrix.diagonal()
apc = pd.DataFrame(index=['anger', 'happy', 'neutral', 'sad', 'suprise'],data=acc_per_class,columns=['Accuracy per class'])
apc = round(apc.iloc[0:]*100,2)
show_plot("Accuracy per class", apc['Accuracy per class'])

In [ ]:
# Overall accuracy of the model
print("Accuracy: ", metrics.accuracy_score(y_test, prediction)*100)

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [True, True])
cm_display.plot()
plt.show()

#from 0 to 4: anger, happy, neutral, sad, suprise